In [545]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [546]:
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [547]:
import pandas as pd

In [548]:
titanic = pd.read_csv('train.csv')

In [549]:
test = titanic.copy()

In [550]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [551]:
print(titanic.info(), titanic.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None (891, 12)


### titanic['PassengerId']　は nd.arrayだが
### titanic[['PassengerId']]　とすると pd.DataFrame　になる

### columnsは titanic.PassengerId　で表示できるとは

In [552]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(titanic, test_size=0.2, random_state=42)
train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)

In [553]:
titanic = train_set.copy()

In [554]:
titanic_labels = titanic['Survived'].copy()
titanic = titanic.drop('Survived', axis=1)
test_labels = test_set['Survived'].copy()
test_set = test_set.drop('Survived', axis=1)

In [555]:
titanic = titanic.drop('Cabin', axis=1)
titanic = titanic.drop('PassengerId', axis=1)
titanic = titanic.drop('Name', axis=1)
test_set = test_set.drop('Cabin', axis=1)
test_set = test_set.drop('PassengerId', axis=1)
test_set = test_set.drop('Name', axis=1)

## dropはこれでできるよ
'df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)'

In [556]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [557]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

class for_Sex_and_Embarked(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        titanic_embarked_encoded, titanic_embarked_categories = X[self.attribute_names[1]].factorize()
        imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
        imp_embarked = imp.fit_transform(titanic_embarked_encoded.reshape(-1,1))
        
        titanic_sex_encoded, titanic_sex_categories = X[self.attribute_names[0]].factorize()
        imp_sex = imp.fit_transform(titanic_sex_encoded.reshape(-1,1))
        
        encoder = OneHotEncoder()
        embarked_1hot = encoder.fit_transform(imp_embarked).toarray()
        sex_1hot = encoder.fit_transform(imp_sex).toarray()
        
        return np.c_[embarked_1hot, sex_1hot]

## こんなに簡単に数値変換できるとは！
~~~ruby
`df.Embarked = df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
df.Cabin = df.Cabin.replace('NaN', 0)
df.Sex = df.Sex.replace(['male', 'female'], [0, 1])
df.Age = df.Age.replace('NaN', 0)`
~~~

In [558]:
import numpy as np

## Series　引数。文字列は捨てて、数値だけをSeriesで返す関数。
def ticket_modify(T):
    for i in range(len(T)):
        if len(T[i]) == 2:
            T[i] = T[i][1]
        else:
            T[i] = T[i][0]
    return T

In [559]:
def StrToInt(T):
    for i in range(len(T)):
        if T[i].isdigit():
            T[i] = int(T[i])
        else:
            T[i] = np.nan
    imp_median = Imputer(missing_values='NaN', strategy='median')
    T = imp_median.fit_transform(T.values.reshape(-1,1))
    return T

In [560]:
class forTicket(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        ticket_split = X[self.attribute_names[0]].apply(lambda x: x.split())
        ticket_modified = ticket_modify(ticket_split)
        X = StrToInt(ticket_modified)
        return X

In [561]:
class just_to_values(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.values

In [562]:
cat_Fare = ['Fare']
cat_age = ['Age']
cat_sex_embarked = ['Sex', 'Embarked']
cat_ticket = ['Ticket']
cat_others = ['Pclass', 'SibSp', 'Parch']

In [563]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

fare_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_Fare)),
        ('imp', Imputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

age_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_age)),
    ('imp', Imputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

sex_embarked_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_sex_embarked)),
    ('imp', for_Sex_and_Embarked(cat_sex_embarked))
])

ticket_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_ticket)),
    ('imp', forTicket(cat_ticket)),
    ('std_scaler', StandardScaler())
])

others_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_others)),
    ('imp', just_to_values())
])

In [564]:
from sklearn.pipeline import  FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ('fare_pipeline',fare_pipeline),    
    ('age_pipeline',age_pipeline),
    ('sex_embarked_pipeline',sex_embarked_pipeline),
    ('ticket_pipeline',ticket_pipeline),
    ('others_pipeline',others_pipeline)
])

In [565]:
titanic_prepared = full_pipeline.fit_transform(titanic)
test_prepared = full_pipeline.fit_transform(test_set)

In [566]:
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,male,45.5,0,0,113043,28.5000,S
1,2,male,23.0,0,0,28425,13.0000,S
2,3,male,32.0,0,0,STON/O 2. 3101293,7.9250,S
3,3,male,26.0,1,0,350025,7.8542,S
4,3,female,6.0,4,2,347082,31.2750,S


In [567]:
pd.DataFrame(titanic_prepared).head()

,0,1,2,3,4,5,6,7,8,9,10
0,-0.078684,1.253641,1.0,0.0,0.0,1.0,0.0,-0.258897,1.0,0.0,0.0
1,-0.377145,-0.477284,1.0,0.0,0.0,1.0,0.0,-0.406763,2.0,0.0,0.0
2,-0.474867,0.215086,1.0,0.0,0.0,1.0,0.0,-0.258885,3.0,0.0,0.0
3,-0.476230,-0.246494,1.0,0.0,0.0,1.0,0.0,0.155218,3.0,1.0,0.0
4,-0.025249,-1.785093,1.0,0.0,0.0,0.0,1.0,0.150075,3.0,4.0,2.0


In [568]:
titanic_prepared.shape

(712, 11)

In [569]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(titanic_prepared, titanic_labels)

/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [570]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, titanic_prepared, titanic_labels, cv=3, scoring="accuracy")

/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), Fut

array([0.75630252, 0.80168776, 0.75949367])

In [571]:
from sklearn.base import BaseEstimator

class All_died(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [572]:
all_died = All_died()
cross_val_score(all_died, titanic_prepared, titanic_labels, cv=3, scoring='accuracy')

array([0.59663866, 0.63291139, 0.64135021])

In [573]:
from sklearn.model_selection import cross_val_predict

predict_titanic = cross_val_predict(sgd_clf, titanic_prepared, titanic_labels, cv=3)

/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), Fut

In [574]:
from sklearn.metrics import confusion_matrix
confusion_matrix(titanic_labels, predict_titanic)

array([[374,  70],
       [ 92, 176]])

In [575]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(titanic_prepared, titanic_labels)
cross_val_score(forest_clf, titanic_prepared, titanic_labels, cv=3, scoring="accuracy")

array([0.79411765, 0.83122363, 0.80168776])

In [576]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(titanic_prepared, titanic_labels)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomFor...f',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [577]:
test_set.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,male,NaN,1,1,2661,15.2458,C
1,2,male,31.0,0,0,C.A. 18723,10.5000,S
2,3,male,20.0,0,0,SOTON/O2 3101287,7.9250,S
3,2,female,6.0,0,1,248727,33.0000,S
4,3,female,14.0,1,0,2651,11.2417,C


In [578]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(titanic_prepared, titanic_labels)
    test_pred = clf.predict(test_prepared)
    print(clf.__class__.__name__, accuracy_score(test_labels, test_pred))

LogisticRegression 0.7877094972067039
RandomForestClassifier 0.7653631284916201
SVC 0.8044692737430168
VotingClassifier 0.8100558659217877


/Users/keisuke/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [579]:
import pandas as pd

In [580]:
a = pd.read_csv('train.csv')
b = pd.read_csv('test.csv')

In [581]:
a.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [582]:
a_labels = a['Survived'].copy()
a.drop(['Survived', 'PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)

b.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)

just_train = full_pipeline.fit_transform(a)
final_set = full_pipeline.fit_transform(b)

In [583]:
from sklearn.svm import SVC
svm_clf = SVC(random_state=42)
svm_clf.fit(just_train, a_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [584]:
predict = svm_clf.predict(final_set)
predict

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [589]:
# テスト値を再読み込みして，SVMでクラス分類したカラムを追加
df_out = pd.read_csv("test.csv")
df_out["Survived"] = predict

# outputディレクトリに出力する
df_out[["PassengerId","Survived"]].to_csv("submission.csv",index=False)

In [590]:
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,male,45.5,0,0,113043,28.5000,S
1,2,male,23.0,0,0,28425,13.0000,S
2,3,male,32.0,0,0,STON/O 2. 3101293,7.9250,S
3,3,male,26.0,1,0,350025,7.8542,S
4,3,female,6.0,4,2,347082,31.2750,S
